In [8]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from wordcloud import WordCloud
from konlpy.tag import Twitter
from collections import Counter
from nltk.probability import FreqDist
from sklearn.preprocessing import QuantileTransformer


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import time
import warnings
warnings.filterwarnings("ignore")

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 14

In [9]:
# import scipy.sparse as spr

---
# 데이터 불러오기

In [10]:
# train (playlist)
train = pd.read_json('data/train.json')

#genre
genre_gn_all = pd.read_json('data/genre_gn_all.json', typ='series')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})

# songs
song_meta = pd.read_json('data/song_meta.json')

# plylst 별 tag 유사도를 살펴보자


> - 1) `train`, `train['tags']` 확인   
> - 2) `train['tags']` [ ] 제거
> - 3) plylst 좋아요 5개 이상 받은 plylst의 태그 선별 (11만 개 -> 3만3천개)
> - 4) Tfidf Vectorizing 사용

## 전처리
데이터 내부엔 list(dict()) 형태로 구성되어 있습니다. 또한 이 안에는 **문자열** 형태로 들어가있구요.  
이를 처리하기 위해서 ast의 literal_eval 함수를 사용했습니다.   

>개인공부   
> `AST 모듈`은 문법을 구조화 시켜주는 모듈    
python 에서 제공하는 기본 type 정도만 변환해주는 용도로 사용 가능   
`literal_eval`은 python 의 기본 자료형 정도만 evaluate 가 가능하도록 지원한다. eval 과 비교해 훨씬 엄격   
그 사용 용도가 eval 대비 훨씬 제한적     
literal_eval 의 용도 :  string을 자료형으로 구조화 시키는 것.

> `apply 함수` :    
numpy의 단일함수를 호출 시 axis=0, 1의 결과값이 같음.   
numpy의 집계함수를 호출 시 axis=0 (행), axis=1 (열)

> `lambda x 함수` : 

In [4]:
# train 정보 확인.
train.info()
train.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115071 non-null  object
 1   id            115071 non-null  int64 
 2   plylst_title  115071 non-null  object
 3   songs         115071 non-null  object
 4   like_cnt      115071 non-null  int64 
 5   updt_date     115071 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.3+ MB


,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000


In [5]:
pd.set_option('display.max_colwidth', -1)

In [6]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 139541, 351214, 650298, 531057, 205238, 706183, 127099, 660493, 461973, 121455, 72552, 223955, 324992, 50104]",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 244277, 461062, 696302, 442765, 532114, 586541, 33389, 244000, 692078, 37741, 645653, 571802, 200183, 61435, 204499, 41749, 129258, 413920, 117205, 6546, 152422, 602724, 425946, 173634, 631268, 409869, 97749, 395416, 103741, 181101, 472144, 414721, 75801, 315216, 192882, 383960, 548636]",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598, 233195, 666852, 686560, 556426, 142974, 331878, 195141, 32017, 617795, 396532, 623704, 516930, 300104, 176874, 443513, 471385, 35784, 153029, 336743, 203558, 348801, 454550]",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스, 겨울왕국, 크리스마스송]",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 100335, 556301, 655561, 534818, 695032, 516602, 521739, 97057, 703323, 295250, 25155, 24275, 273672, 334095, 284990, 679582, 664342, 637135, 68528, 243754, 417111, 414226, 338078, 384299, 542735, 457519, 453762, 349398, 631142, 406082, 548389, 205179, 567076]",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657, 86875, 224139, 14156, 555901, 144617, 134974, 503552, 583828, 566439, 312624, 61595, 643315, 335893, 199192, 587291, 398171, 359101, 84285, 324136, 430005, 664555, 569867, 598239, 568089, 102889, 311997, 664191, 402784, 487106, 327354, 231154, 402984, 684625, 63146, 343677, 360131, 583375, 181670, 314344, 617473, 33244, 635753, 157283, 250477, 405687, 146266, 371498]",9,2011-10-25 13:54:56.000


In [7]:
# train tags 값 확인
train['tags']

0         [락]                                                         
1         [추억, 회상]                                                    
2         [까페, 잔잔한]                                                   
3         [연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스, 겨울왕국, 크리스마스송]
4         [댄스]                                                        
          ...                                                         
115066    [록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]                        
115067    [일렉]                                                        
115068    [담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]                      
115069    [잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]                               
115070    [노래추천, 팝송추천, 팝송, 팝송모음]                                      
Name: tags, Length: 115071, dtype: object

> 개인공부   
> map과 apply    
> map = 컬럼 하나의 행에만 적용      
apply = 모든 행에 대해 적용   

## tags의 [ ] 제거한 column(tags_string) 생성

In [8]:
# tags의 [ ] 제거한 column(tags_string) 생성
train['tags_string'] = train['tags'].apply(lambda x: ' '.join(x)) 
train.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_string
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 139541, 351214, 650298, 531057, 205238, 706183, 127099, 660493, 461973, 121455, 72552, 223955, 324992, 50104]",71,2013-12-19 18:36:19.000,락


## train_tagup5 변수 선언 :
- tags_cnt 태그 개수 카운트 
- 태그가 5개 이상 달린 plylst으로 이뤄진 DF변수 생성
>TfidfVectorizer을 태그 기반으로 수행하려고 했지만 allocate 용량이 너무 커서 에러남.    
플레이리스트 좋아요를 5개 이상 받은 플레이리스트로 범위를 줄여봄    
그 결과 11만개에서 3만3천개로 줄여짐.   

In [9]:
# train_tagup5 변수 선언 : 태그가 5개 이상 달린 plylst으로 이뤄진 DF변수 생성
train['tags_cnt'] = train['tags'].map(lambda x : len(x)) # tag_cnt 변수 생성
train_tagup5 = train.query('tags_cnt >5') # 태그가 5개 이상 달린 플레이리스트 선별
print(train_tagup5.shape) # plylist를 11만개에서 3만3천개로 크기 줄임.
train_tagup5.head(1)

(33476, 8)


,tags,id,plylst_title,songs,like_cnt,updt_date,tags_string,tags_cnt
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스, 겨울왕국, 크리스마스송]",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 100335, 556301, 655561, 534818, 695032, 516602, 521739, 97057, 703323, 295250, 25155, 24275, 273672, 334095, 284990, 679582, 664342, 637135, 68528, 243754, 417111, 414226, 338078, 384299, 542735, 457519, 453762, 349398, 631142, 406082, 548389, 205179, 567076]",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송,10


In [10]:
train['tags_cnt'].sum() # 총 태그 수 476331개(중복 포함)

476331

---
## train_tagup5 DF확인

In [11]:
print(train_tagup5.columns)
display(train_tagup5.head(1))

Index(['tags', 'id', 'plylst_title', 'songs', 'like_cnt', 'updt_date',
       'tags_string', 'tags_cnt'],
      dtype='object')


,tags,id,plylst_title,songs,like_cnt,updt_date,tags_string,tags_cnt
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스, 겨울왕국, 크리스마스송]",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 100335, 556301, 655561, 534818, 695032, 516602, 521739, 97057, 703323, 295250, 25155, 24275, 273672, 334095, 284990, 679582, 664342, 637135, 68528, 243754, 417111, 414226, 338078, 384299, 542735, 457519, 453762, 349398, 631142, 406082, 548389, 205179, 567076]",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송,10


---
## TF-IDF 벡터화

전처리한 데이터를 TF-IDF 방법을 이용해 벡터로 변환.  

> `단어 문서 행렬`(term-document matrix: 이하 `TDM`)   
`tfidf` 뒤에 `.get_feature_names()` 메써드를 부르면 TDM에 사용된 feature 단어 목록을 볼 수 있다.


In [12]:
tfidf_vector = TfidfVectorizer() 

# tfidf에 fit
tfidf_matrix = tfidf_vector.fit_transform(train_tagup5['tags_string']).toarray() # 태그 단어들을 array로 변환

# TDM에 사용된 feature 단어 목록을 볼 수 있다.
tfidf_matrix_feature = tfidf_vector.get_feature_names() 

In [13]:
tfidf_matrix # 벡터화된 태그(tags_cnt 5 이상 plylst의 태그벡터)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
# tfidf_matrix_feature # tags_cnt 5 이상 plylst 속 tags

['00',
 '007',
 '007시리즈',
 '00s',
 '00년',
 '00년대',
 '00년대노래',
 '00년대발라드',
 '01',
 '015b',
 '02',
 '03',
 '03월',
 '04',
 '05',
 '0515',
 '06',
 '07',
 '0channel',
 '0개국어',
 '0살',
 '0시',
 '10',
 '100',
 '100000',
 '1000만관객동원',
 '100곡',
 '100위',
 '101',
 '10cm',
 '10곡',
 '10년간의_기록',
 '10년대',
 '10년전',
 '10대',
 '10대가10대에게',
 '10대감성',
 '10대들의우상',
 '10만개',
 '10분전',
 '10센치',
 '10월',
 '10월4주차',
 '10월음악',
 '10주',
 '10주년',
 '10초',
 '10화',
 '11',
 '110bpm',
 '11곡',
 '11월',
 '11월11일',
 '11월1일',
 '11월1주차',
 '11월3주차',
 '11월4주차',
 '11월_2주차',
 '11월_3주차',
 '11월_컴백',
 '11월넷째주',
 '11월둘째주',
 '11일',
 '11화',
 '12',
 '1218',
 '1225',
 '1248',
 '128bpm',
 '12월',
 '12월1주차',
 '12월25일',
 '12월2주차',
 '12월3주차',
 '12월_2주차',
 '12월둘째주',
 '12월셋째주',
 '12화',
 '13',
 '13명의',
 '13월',
 '13학번',
 '13화',
 '14',
 '1415',
 '1485',
 '14화',
 '14회',
 '15',
 '16',
 '16강',
 '16년_12월',
 '17년_10월',
 '17년_1월',
 '17년_3월',
 '17년_5월',
 '17주년',
 '18',
 '1855',
 '18번',
 '19',
 '1900년',
 '190113',
 '191229',
 '191231',
 '1920',
 '1930',
 '1930

In [21]:
# 행 = plylst_title, 열 = (tags_cnt > 5)인 tags
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = train_tagup5['plylst_title']) # 열 = 장르이름 | 행 = 영화이름
print(tfidf_matrix.shape) # tfidf_matrix 형태 : 행 = 플레이리스트,  열 = 태그 벡터
tfidf_matrix.head(2)

(33476, 21121)


,00,007,007시리즈,00s,00년,00년대,00년대노래,00년대발라드,01,015b,...,힙합페스티벌,힙합플레이리스트,힙합플레이야,힙합플레이야2018,힙합홍수,힙해,힙힙힙,힛뎀포크,힛뎀폭,힛뎀폭스
plylst_title,,,,,,,,,,,,,,,,,,,,,
크리스마스 분위기에 흠뻑 취하고 싶을때,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017 Pop Trend,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

## 유사도 구하기

이렇게 만들어진 `tf-idf vector`를 `코사인 유사도`를 활용해서 유사도 값을 구해줍니다.  
이렇게 하면 `plylst`의 개수(n)만큼 n x n의 matirx 형태가 나오게 됩니다.

In [24]:
# 코사인 유사도 구하기
# %%time
tfidf_matrix
cosine_sim = cosine_similarity(tfidf_matrix)

MemoryError: Unable to allocate 5.27 GiB for an array with shape (33476, 21121) and data type float64

In [ ]:
cosine_sim

In [ ]:
cosine_sim.shape

In [ ]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = train_tagup5['plylst_title'], columns = train_tagup5['plylst_title']) # 행 열 모두 영화 이름
print(cosine_sim_df.shape)
cosine_sim_df.head()

In [ ]:
# 1. title series 가져오기
recom_idx = cosine_sim_df.loc[:, '2017 Pop Trend']
print(recom_idx)

In [ ]:
# 2. title values 가져오기
recom_idx = cosine_sim_df.loc[:, '크리스마스 분위기에 흠뻑 취하고 싶을때'].values
print(recom_idx)
print(type(recom_idx))
print(recom_idx.shape)

In [ ]:
# 3. reshape 1 x n 행렬로 변환
recom_idx = cosine_sim_df.loc[:, '크리스마스 분위기에 흠뻑 취하고 싶을때'].values.reshape(1, -1) 
print(recom_idx)
print(type(recom_idx))
print(recom_idx.shape)

In [ ]:
# 4. argsort 코사인 유사도가 높은 기준으로 내림차순 정렬
recom_idx = cosine_sim_df.loc[:, '크리스마스 분위기에 흠뻑 취하고 싶을때'].values.reshape(1, -1).argsort()[:, ::-1] 
print(recom_idx)
print(type(recom_idx))
print(recom_idx.shape)

In [ ]:
# 5 flatten '크리스마스 분위기에 흠뻑 취하고 싶을때'와 코사인 유사도가 높은 순으로 plylst 벡터 10개 array만들기
recom_idx = cosine_sim_df.loc[:, '크리스마스 분위기에 흠뻑 취하고 싶을때'].values.reshape(1, -1).argsort()[:, ::-1].flatten()[0:10] 
print(recom_idx)
print(type(recom_idx))
print(recom_idx.shape)

In [ ]:
# 6 recom_ply_title 추천 plylst 10개 보여주기
recom_ply_title = train_tagup5.iloc[recom_idx, :]
display(recom_ply_title)
print(type(recom_ply_title))

In [ ]:
# 추천된 10개 plylst title만 가져오기
recom_ply_title = train_tagup5.iloc[recom_idx, :].plylst_title
print(type(recom_ply_title))
print(recom_ply_title)

In [ ]:
# 추천된 plylst 리스트로 묶어주기
recom_ply_title = train_tagup5.iloc[recom_idx, :].plylst_title.values
print(type(recom_ply_title))
print(recom_ply_title)

In [ ]:
# recom_tag 변수 생성 
recom_tag = train_tagup5.iloc[recom_idx, :]
display(recom_tag.head(2))

In [ ]:
# recom_tag 변수 태그모음(tags_string)으로 변경
recom_tag = train_tagup5.iloc[recom_idx, :].tags_string
display(recom_tag)

In [ ]:
# 태그모음(tags_string)을 list -> array로 변경
recom_tag = train_tagup5.iloc[recom_idx, :].tags_string.values
display(recom_tag)

In [ ]:
target_ply_title = np.full(len(range(10)), '크리스마스 분위기에 흠뻑 취하고 싶을때')
target_ply_title

In [ ]:
target_tags_list = np.full(len(range(10)), train[train.plylst_title == '크리스마스 분위기에 흠뻑 취하고 싶을때'].tags_string.values)
target_tags_list

## 위 결과값을 하나의 DF로 표현하려고함.

---
# word_2_vec으로 도전

- 단어를 벡터시키는 작업
---
---
##  &#128218; 1.3. 예측 기반 임베딩  Word Embedding
- 실수벡터로 사용할 수 있는. 워드임베딩


In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
tag_sentences = train['tags'].tolist()
model  = Word2Vec(tag_sentences, size=20, window=3, min_count=1)

In [ ]:
model.save('model.w2v')
tag_stc_model = Word2Vec.load('model.w2v')

tag_stc_model.wv['드라이브']

In [ ]:
tag_stc_model.wv.most_similar('샤워') # 희소성배열..
# 다변량으로.. anova 분석기법 

In [ ]:
print(tag_stc_model.wv.vocab.keys())
print(tag_stc_model.wv['아이유'])

In [ ]:
for v in tag_stc_model.wv.vocab.keys() :
    print(v, tag_stc_model.wv[v])
    
x = np.array([tag_stc_model.wv[v] for v in tag_stc_model.wv.vocab.keys()])
print(x.shape)

## 그래프를 통한 시각화

In [ ]:
plt.plot(x[:,10], x[:,10], '*' )

In [ ]:
label = list(tag_stc_model.wv.vocab.keys())
print(label[0])
label[0], x[0,:]

In [ ]:
for i in range(20) : 
    
    plt.scatter(x[i,0], x[i,1], marker='*', color='red')
    plt.text( x[i,0]+0.01, x[i, 1]+0.01, label[i], fontsize=10)

## TSNE를 이용한 word2vec 시각화

In [ ]:
train['tags_string'] = train['tags'].apply(lambda x: ' '.join(x))
tag_stcs= train['tags_string'].tolist()
tag_stcs

In [ ]:
tag_sentences[1].count('추억')

In [ ]:
tag_list = []
for i in tag_stcs:
    tag_list.extend(i)
    
print(tag_list[0:10])

In [ ]:
id_to_tag = {}

for i, v in enumerate(tag_sentences):
    id_to_tag[i] = v

type(id_to_tag[0])
id_to_tag[0]


In [ ]:
tag_to_id = {v:k  for k, v in id_to_tag.items()}


In [ ]:
tag_sentences

In [ ]:
from sklearn.manifold import TSNE

model  = Word2Vec(tag_sentences['락','발라드'], size=20, window=3, min_count=1)

x = np.array([model.wv[v] for v in model.wv.vocab.keys()])

label = list(model.wv.vocab.keys())

tsne = TSNE(n_components = 2)
x_sne = tsne.fit_transform(x)

print(x_sne.shape)


In [ ]:
for p, l in zip(x_sne, label) :
    plt.scatter( p[0], p[1], marker='*', color='red')
    plt.text( p[0]+0.01, p[1]+0.01, l, fontsize=10)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
x_pca = pca.fit_transform(x)
print(x_pca.shape)

for p, l in zip(x_pca, label):
    plt.scatter( p[0], p[1], marker='*', color='blue')
    plt.text( p[0]+0.01, p[1]+0.01, l, fontsize=10)